## 1st level. TensorFlow Speech Recognition Challenge

- [필사 정리본](https://goodday-lab.tistory.com/4)
- 정리: [1](https://log-laboratory.tistory.com/323), [2](https://log-laboratory.tistory.com/324?category=1104438)

In [ ]:
# system
import os

# raw data
import pandas as pd
import numpy as np

from scipy import signal
from scipy.io import wavfile
from scipy.fftpack import fft
import librosa

# modeling
from sklearn.decomposition import PCA

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as ipd
import librosa.display
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

In [ ]:
!pip install py7zr
import py7zr

In [ ]:
# path = "../input/tensorflow-speech-recognition-challenge/"
# with py7zr.SevenZipFile(path + "train.7z", 'r') as z:
#     z.extractall(path="/kaggle")
# with py7zr.SevenZipFile(path + "test.7z", 'r') as z:
#     z.extractall(path="/kaggle")

In [ ]:
for dirname, _, filenames in os.walk('/kaggle'): 
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

## 1. Visualization of recordings - input features

인간의 청각에는 두 가지 이론이 있다. 장소와 시간은 음성인식에서 중요한 영향으로 본다. 스펙트럼 분석(주파수) 입력 및 보다 정교한 feature MFCC(Mel Frequency Cepstral Coefficients)

### 1.1 Wave and spectrogram

In [ ]:
train_audio_path = "../train/audio/"
filename = "yes/0a7c2a8d_nohash_0.wav"
sample_rate, samples = wavfile.read(str(train_audio_path) + filename)

spectrogram을 계산하는 함수를 정의한다.<br />
참고로, 우리는 spectrogram 값의 log를 취하고 있다.<br />
그것은 우리의 plot을 훨씬 더 명확하게 할 것이고, 더욱이 그것은 사람들이 듣는 방식과 엄격히 연결되어 있다.<br />
log에 대한 입력으로 0 값이 없다는 보장이 필요하다.

In [ ]:
def log_specgram(audio, sample_rate, window_size=20, step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, times, spec = signal.spectrogram(audio, fs=sample_rate, window="hann",
                                            nperseg=nperseg, noverlap=noverlap,
                                            detrend=False)
    return freqs, times, np.log(spec.T.astype(np.float32) + eps)

Nyquist theorem에 따르면, 주파수 범위는 0 ~ 8000 정도 된다.

In [ ]:
freqs, times, spectrogram = log_specgram(samples, sample_rate)

fig, ax = plt.subplots(2, 1, figsize=(14, 8))

ax[0].plot(np.linspace(0, sample_rate / len(samples), sample_rate), samples)
ax[0].set_ylabel("Ampliude")
ax[0].set_title("Raw wave of " + filename)

ax[1].imshow(spectrogram.T, aspect="auto", origin="lower",
             extent=[times.min(), times.max(), freqs.min(), freqs.max()])
ax[1].set_xticks(times[::16])
ax[1].set_xlabel("Freqs in Hz")
ax[1].set_yticks(freqs[::16])
ax[1].set_ylabel("seconds")
ax[1].set_title("Spectrogram of " + filename)

plt.show()

In [ ]:
mean = np.mean(spectrogram, axis=0)
std = np.std(spectrogram, axis=0)
spectrogram = (spectrogram - mean) / std

### 1.2 MFCC

MFCC는 인간의 청력 특성을 모방할 준비가 잘 되어 있다는 것을 알 수 있다고 설명했다. librosa python package를 사용하여 Mel power spectrogram과 MFCC를 계산할 수 있다.

In [ ]:
librosa_samples, librosa_sample_rate = librosa.load(str(train_audio_path) + filename)
S = librosa.feature.melspectrogram(librosa_samples, sr=librosa_sample_rate, 
                                   n_mels=128, fmax=8000)
log_S = librosa.power_to_db(S, ref=np.max)

In [ ]:
plt.figure(figsize=(12, 4))
librosa.display.specshow(log_S, sr=sample_rate, x_axis="time", y_axis="mel")
plt.colorbar(format="%+02.0f dB")
plt.title("Mel Power spectrogram")
plt.tight_layout()
plt.show()

In [ ]:
mfcc = librosa.feature.mfcc(S=log_S, n_mfcc=13)
delta2_mfcc = librosa.feature.delta(mfcc, order=2)

In [ ]:
plt.figure(figsize=(12, 4))
librosa.display.specshow(delta2_mfcc)
plt.colorbar()
plt.xlabel("Time")
plt.ylabel("MFCC coeffs")
plt.title("MFCC")
plt.tight_layout()
plt.show()

### 1.3 Sprectrogram in 3d

In [ ]:
data = [go.Surface(x=times, y=freqs, z=spectrogram.T)]
layout = go.Layout(
    title='Spectrogram of "yes" in 3d',
    scene=dict(xaxis=dict(title="Time", range=[times.min(), times.max()],),
               yaxis=dict(title="Frequency", range=[freqs.min(), freqs.max()]),
               zaxis=dict(title="Log amplitude"),),)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

### 1.4 Silence removal

In [ ]:
ipd.Audio(samples, rate=sample_rate)

In [ ]:
samples_cut = samples[4000:13000]
ipd.Audio(samples_cut, rate=sample_rate)

'y', 'e', 's'의 중심?으로 예측되는 곳을 표시해보자.

In [ ]:
freqs, times, spectrogram_cut = log_specgram(samples_cut, sample_rate)

fig, ax = plt.subplots(2, 1, figsize=(14, 8))

ax[0].plot(samples_cut)
ax[0].set_ylabel("Amplitude")
ax[0].set_title("Raw wave of " + filename)

ax[1].imshow(spectrogram_cut.T, aspect="auto", origin="lower",
             extent=[times.min(), times.max(), freqs.min(), freqs.max()])

ax[1].set_xticks(times[::16])
ax[1].set_xlabel("Samples")
ax[1].set_yticks(freqs[::16])
ax[1].set_ylabel("Frequencies * 0.1")
ax[1].set_title("Spectrogram of " + filename)

ax[1].text(0.06, 1000, "Y", fontsize=18)
ax[1].text(0.17, 1000, "E", fontsize=18)
ax[1].text(0.36, 1000, "S", fontsize=18)

xcoords = [0.025, 0.11, 0.23, 0.49]
for xc in xcoords:
    ax[0].axvline(x=xc * samples_cut.shape[0], c='r')
    ax[1].axvline(x=(xc * samples_cut.shape[0]) / sample_rate, c='r')

plt.show()

### 1.5 Resampling - dimensionality reductions

데이터의 치수성을 줄이는 또 다른 방법은 기록을 다시 샘플링하는 것이다.<br />
녹음은 16K 주파수로 샘플링되기 때문에 자연스럽지 않다는 것을 들 수 있고, 우리는 보통 훨씬 더 많이 듣는다. 그러나 가장 많은 음성 관련 주파수는 더 작은 대역으로 제시된다. 그렇기 때문에 당신은 GSM 신호가 8000Hz로 샘플링되는 전화와 통화하는 다른 사람을 여전히 이해할 수 있다.

요약하면 데이터 집합을 8k로 다시 샘플링할 수 있다. 우리는 중요하지 않아야 할 정보를 버리고 데이터의 크기를 줄일 것이다. 우리는 이것이 위험할 수 있다는 것을 기억해야 한다. 빠른 계산을 위해서 FFT(Fast Fouier Transform)의 계산이 필요하다.

In [ ]:
def custom_fft(y, fs):
    T = 1.0 / fs
    N = y.shape[0]
    yf = fft(y)
    xf = np.linspace(0.0, 1.0 / (2.0 * T), N // 2)
    # FFT는 시뮬레이션을 통해 전반부만 촬영한다.
    # FFT도 복잡해서 진짜 부분만 가져가면 된다. (abs)
    vals = 2.0 / N * np.abs(yf[0:N // 2])
    return xf, vals

- 일단 받아씀

녹음된 내용을 읽고 다시 샘플링한 후 들어보자.<br />
또한 FFT, Notice를 비교할 수 있는데, 원래 신호에는 4000Hz 이상의 정보가 거의 없다.<br />
녹음된 내용을 읽고 다시 샘플링한 후 들어보자.<br />
또한 FFT, Notice를 비교할 수 있는데, 원래 신호에는 4000Hz 이상의 정보가 거의 없다.

In [ ]:
filename = "happy/0b09edd3_nohash_0.wav"
new_sample_rate = 8000
sample_rate, samples = wavfile.read(str(train_audio_path) + filename)
resampled = signal.resample(samples, int(new_sample_rate / sample_rate * samples.shape[0]))

In [ ]:
ipd.Audio(samples, rate=sample_rate)

In [ ]:
ipd.Audio(resampled, rate=new_sample_rate)

귀가 좋은 지인은 둘에서 차이를 찾을 수 있을까? 일단 난 못 찾는다.

In [ ]:
xf, vals = custom_fft(samples, sample_rate)
plt.figure(figsize=(12, 4))
plt.plot(xf, vals)
plt.grid()
plt.xlabel("Frequency")
plt.title("FFT of recording sampled with " + str(sample_rate) + " Hz")
plt.show()

In [ ]:
xf, vals = custom_fft(resampled, new_sample_rate)
plt.figure(figsize=(12, 4))
plt.plot(xf, vals)
plt.grid()
plt.xlabel("Frequency")
plt.title("*Resample data* FFT of recording sampled with " + str(new_sample_rate) + " Hz")
plt.show()

### 1.6 Features extraction steps

변수 추출 알고리즘은 아래와 같이 진행한다.

1. Resampling
2. VAD (Voice Activity Detection)
3. 신호 길이가 같도록 0으로 padding
4. Log spectrogram (or MFCC, or PLP)
5. 평균 및 표준 feature 정규화
7. 임시 정보를 얻기 위해 지정된 수의 프레임 쌓기

중요도가 높은 건 아니지만, 이를 이용하면 약간의 왜곡을 발견할 수 있다.

## 2. Dataset investigation

### 2.1 Number of files

In [ ]:
dirs = [f for f in os.listdir(train_audio_path)
        if os.path.isdir(os.path.join(train_audio_path, f))]
dirs.sort()
print("Number of labels", str(len(dirs)))

In [ ]:
dirs

In [ ]:
number_of_recordings = []
for direct in dirs:
    waves = [f for f in os.listdir(os.path.join(train_audio_path, direct))
             if f.endswith(".wav")]
    number_of_recordings.append(len(waves))

In [ ]:
data = [go.Histogram(x=dirs, y=number_of_recordings)]
trace = go.Bar(x=dirs, y=number_of_recordings,
               marker=dict(color=number_of_recordings, colorscale="ylorrd", showscale=True))
layout = go.Layout(xaxis=dict(title="Words"), yaxis=dict(title="Number of recordings"),
                   title="Number of recordings in given label")

py.iplot(go.Figure(data=[trace], layout=layout))

적당히 균등하게 보이기는 하는데, noise가 있네?

### 2.2 Deeper into recordings

녹음은 매우 다른 출처에서 나온다. 일부는 이동형 GSM 채널에서 올 수 있다.<br />
그럼에도 불구하고, 한 명의 발화자가 test와 train data로 분리되지 않는 것이 매우 중요하다. 이 두 가지 시험 유형을 보고 들어 보자. <- ???

In [ ]:
filenames = ["on/004ae714_nohash_0.wav", "on/0137b3f4_nohash_0.wav"]
for filename in filenames:
    sample_rate, samples = wavfile.read(str(train_audio_path) + filename)
    xf, vals = custom_fft(samples, sample_rate)
    
    plt.figure(figsize=(12, 4))
    plt.plot(xf, vals)
    plt.grid()
    plt.xlabel("Frequency")
    plt.title("FFT of speaker " + filename[4:11])
    plt.show()

들어보자

In [ ]:
ipd.Audio(os.path.join(train_audio_path, filenames[0]))

In [ ]:
ipd.Audio(os.path.join(train_audio_path, filenames[1]))

In [ ]:
filename = "yes/01bb6a2a_nohash_1.wav"
sample_rate, samples = wavfile.read(str(train_audio_path) + filename)
ipd.Audio(samples, rate=sample_rate)

In [ ]:
freqs, times, spectrogram = log_specgram(samples, sample_rate)

plt.figure(figsize=(10, 7))
plt.imshow(spectrogram.T, aspect="auto", origin="lower",
           extent=[times.min(), times.max(), freqs.min(), freqs.max()])

plt.xticks(times[::16])
plt.xlabel("Time")
plt.yticks(freqs[::16])
plt.ylabel("Freqs")
plt.title("Spectrogram of " + filename)
plt.show()

앞에 애매한 침묵이 있는데 압축 과정에서 문제가 생긴 것으로 판단된다.<br />
즉, 우리는 매우 특정한 음향 환경에 과도하게 적응하는 것을 막아야 한다.

### 2.3 Recordings length

모든 파일이 1초 정도인지 확인해보자.

In [ ]:
num_of_shorter = 0
for direct in dirs:
    waves = [f for f in os.listdir(os.path.join(train_audio_path, direct))
             if f.endswith(".wav")]
    for wav in waves:
        sample_rate, samples = wavfile.read(train_audio_path + direct + "/" + wav)
        if samples.shape[0] < sample_rate:
            num_of_shorter += 1
print("Number of recordings shorter than 1 second:", num_of_shorter)

### 2.4 Mean spectrograms and fft

모든 단어의 mean FFT plot을 보자.

In [ ]:
to_keep = "yes no up down left right on off stop go".split()
dirs = [d for d in dirs if d in to_keep]
print(dirs)

In [ ]:
for direct in dirs:
    vals_all = []; spec_all = []
    waves = [f for f in os.listdir(os.path.join(train_audio_path, direct))
             if f.endswith(".wav")]
    for wav in waves:
        sample_rate, samples = wavfile.read(train_audio_path + direct + "/" + wav)
        if samples.shape[0] != 16000: continue
        
        xf, vals = custom_fft(samples, 16000)
        vals_all.append(vals)
        
        freqs, times, spec = log_specgram(samples, 16000)
        spec_all.append(spec)
    
    fig, ax = plt.subplots(1, 2, figsize=(14, 4))
    ax[0].plot(np.mean(np.array(vals_all), axis=0))
    ax[0].grid()
    ax[0].set_title("Mean fft of " + direct)
    
    ax[1].imshow(np.mean(np.array(spec_all), axis=0).T, aspect="auto", origin="lower",
                 extent=[times.min(), times.max(), freqs.min(), freqs.max()])
    ax[1].set_xticks(times[::16])
    ax[1].set_yticks(freqs[::16])
    ax[1].set_title("Mean spectrogram of " + direct)
    plt.show()

### 2.5 Frequency components across the words

In [ ]:
def violinplot_frequency(dirs, freq_ind):
    spec_all = []; ind = 0
    for direct in dirs:
        spec_all.append([])
        
        waves = [f for f in os.listdir(os.path.join(train_audio_path, direct))
                 if f.endswith(".wav")]
        for wav in waves[:100]:
            sample_rate, samples = wavfile.read(train_audio_path + direct + "/" + wav)
            freqs, times, spec = log_specgram(samples, sample_rate)
            spec_all[ind].extend(spec[:, freq_ind])
        ind += 1
    
    minimum = min([len(spec) for spec in spec_all])
    spec_all = np.array([spec[:minimum] for spec in spec_all])
    
    plt.figure(figsize=(13, 7))
    sns.violinplot(data=pd.DataFrame(spec_all.T, columns=dirs))
    plt.xlabel("Words")
    plt.ylabel("Amount of frequency in a word")
    plt.title("Frequency " + str(freqs[freq_ind]) + " Hz")
    plt.show()

In [ ]:
violinplot_frequency(dirs, 20)

In [ ]:
violinplot_frequency(dirs, 50)

In [ ]:
violinplot_frequency(dirs, 120)

### 2.7 Anomaly detection

혹시나 있을 이상 데이터를 감지하고 없애야 한다.

In [ ]:
fft_all = []; names = []
for direct in dirs:
    waves = [f for f in os.listdir(os.path.join(train_audio_path, direct))
             if f.endswith(".wav")]
    for wav in waves:
        sample_rate, samples = wavfile.read(train_audio_path + direct + "/" + wav)
        if samples.shape[0] != sample_rate:
            samples = np.append(samples, np.zeros((sample_rate - samples.shape[0],)))
        
        x, val = custom_fft(samples, sample_rate)
        fft_all.append(val)
        names.append(direct + "/" + wav)

fft_all = np.array(fft_all)
fft_all = (fft_all - np.mean(fft_all, axis=0)) / np.std(fft_all, axis=0)

In [ ]:
pca = PCA(n_components=3)
fft_all = pca.fit_transform(fft_all)

In [ ]:
def interactive_3d_plot(data, names):
    data = go.Data([go.Scatter3d(x=data[:, 0], y=data[:, 1], z=data[:, 2],
                                 mode="markers", text=names)])
    layout = go.Layout(title="Anomaly detection")
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)

In [ ]:
interactive_3d_plot(fft_all, names)

위의 그림에서 떨어져 있는 이상치를 들어보자. 그 중에서도 yes/e4b02540_nohash_0.wav, go/0487ba9b_nohash_0.wav의 포인트들은 중앙으로부터 많이 떨어져있다.

In [ ]:
ipd.Audio(os.path.join(train_audio_path, "go/0487ba9b_nohash_0.wav"))

In [ ]:
ipd.Audio(os.path.join(train_audio_path, "yes/e4b02540_nohash_0.wav"))

지지직 거리는 잡음이 많이 껴 있음을 확인할 수 있다. 만약 개별 단어의 이상치가 궁금하다면 seven 폴더에서 찾아볼 수 있다.

In [ ]:
ipd.Audio(os.path.join(train_audio_path, "seven/e4b02540_nohash_0.wav"))